# Deep Dive in Pytorch
- 파이토치 사용법을 좀 더 고급화 하기 위해, 책 한 권 [딥러닝 파이토치 교과서]을 정해, 정리해보려고 한다.

## 1. Tensor
- Pytorch는 텐서에 대한 제어를 확실히 해야함.
- 개인적으로, Pytorch와 Tensorflow를 병행해서 사용하다보니, 때때로 많이 헷갈리는 경우가 생김.

### 텐서 생성 및 변환

In [9]:
# Tensor 생성
import torch
print(torch.tensor([[1, 2], [3, 4]]))
print(torch.tensor([[1, 2], [3, 4]], device="cuda:0"))
print(torch.tensor([[1, 2], [3, 4]], dtype=torch.float64))

tensor([[1, 2],
        [3, 4]])


AssertionError: Torch not compiled with CUDA enabled

In [3]:
# Tensor -> numpy array
tensor_a = torch.tensor([[1, 2], [3, 4]])
print(tensor_a.numpy())

tensor_gpu = torch.tensor([[1, 2], [3, 4]], device="cuda:0")
print(tensor_gpu.to("cpu").numpy())

# GPU 메모리에 올라가 있는 Tensor는 바로 numpy() 변환 되지 않음
# print(tensor_gpu.numpy())

[[1 2]
 [3 4]]
[[1 2]
 [3 4]]


### 텐서의 자료형 및 인덱스 조작
**Tensor 자료형**
1. torch.FloatTensor : 32비트 부동 소수점 텐서
2. torch.DoubleTensor : 64비트 부동 소수점 텐서
3. torch.LongTensor : 64비트 부호가 있는 정수

In [5]:
float_tensor = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7])
print(float_tensor[0], float_tensor[1], float_tensor[2])
print(float_tensor[0:3], float_tensor[3:5], float_tensor[-4:-1])

tensor(1.) tensor(2.) tensor(3.)
tensor([1., 2., 3.]) tensor([4., 5.]) tensor([4., 5., 6.])


###  텐서 연산
- 텐서간 연산은 텐서의 타입이 다르면 연산이 불가능
- FloatTensor와 DoubleTensor 간에 사칙 연산을 하면 에러 발생
    - pytorch 1.11 에서는 연산 가능

In [7]:
tensor_a = torch.Tensor([1, 2, 3])
print(tensor_a)
tensor_b = torch.tensor([4, 5, 6])
print(tensor_b)

print(tensor_a + tensor_b)

tensor([1., 2., 3.])
tensor([4, 5, 6])
tensor([5., 7., 9.])


In [8]:
sample_float = torch.FloatTensor([2, 4, 6])
sample_double = torch.DoubleTensor([1, 2, 3])

print(sample_float + sample_double)

tensor([3., 6., 9.], dtype=torch.float64)


In [9]:
sample_float

tensor([2., 4., 6.])

In [10]:
sample_double

tensor([1., 2., 3.], dtype=torch.float64)

### 텐서 차원 조작
- 텐서 차원 조작 방법 : ```view```
    - numpy의 ```reshape```과 유사
- 텐서 결합 : ```stack```, ```cat```
    - ```cat``` : 다른 길이의 텐서를 하나로 병합할 때 사용
- 텐서 차원 교환 : ```t```, ```transpose```
    - ```transpose``` : 행렬 전치 외에도, 차원의 순서 변경 때도 사용


In [3]:
tensor_sample = torch.tensor([[1, 2], [3, 4]])

print(tensor_sample.shape)
print("---")
print(tensor_sample.view(4, 1))
print("---")
print(tensor_sample.view(-1))
print("---")
print(tensor_sample.view(1, -1))
print("---")
print(tensor_sample.view(-1, 1))

torch.Size([2, 2])
---
tensor([[1],
        [2],
        [3],
        [4]])
---
tensor([1, 2, 3, 4])
---
tensor([[1, 2, 3, 4]])
---
tensor([[1],
        [2],
        [3],
        [4]])


## 2. Data Preparation
- Pytorch Data 호출 방법
    1. 파이썬 라이브러리 (판다스(Pandas))
    2. Pytorch 제공 데이터
- 데이터 타입에 따른 호출 방법
    1. 이미지
        - 분산된 파일에서 데이터 읽기
        - 전처리
        - 배치 단위로 분할 처리
    2. 텍스트
        - 임베딩 과정
        - 서로 다른 길이의 Sequence를 배치 단위로 분할 처리

### 판다스를 사용한 데이터 임포트

In [10]:
import pandas as pd

data = pd.read_csv('./data/sample_01.csv')

x = torch.from_numpy(data['x'].values).unsqueeze(dim=1).float()
y = torch.from_numpy(data['y'].values).unsqueeze(dim=1).float()

print(x)
print(y)

tensor([[1.],
        [2.],
        [3.]])
tensor([[ 8.],
        [ 9.],
        [10.]])


### 커스텀 데이터셋을 만들어서 사용
- 딥러닝은 대량의 데이터를 이용해 모델 학습을 시킴
- 데이터를 한번에 메모리에 불러와서 훈련시키면 시간과 비용측면에서 비효율
- 데이터를 한번에 다 부르지 않고, 커스텀 데이터셋을 이용

**CustomDataset 구현**
```
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # 필요한 변수 선언, 데이터셋 전처리 함수

    def __len__(self):
        # 데이터셋 길이, 총 샘플의 수를 가져오는 함수

    def __getitem__(self, index):
        # 데이터셋에서 특정 데이터를 가져오는 함수 (index 번째 데이터를 반환 하는 함수)
        # 반환 값은 텐서 형태
```

**DataLoader**
- 데이터로더 (DataLoader) 객체는 학습에 사용될 데이터 전체를 보관.
- 모델 학스비, 배치 크기만큼 데이터를 꺼내서 사용
- DataLoader는 데이터를 미리 잘라 놓는 것이 아니라, iterator를 활용해 index를 이용하여 배치 크기만큼 데이터를 반환

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.label = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        sample = torch.tensor(self.label.iloc[index, 0:3]).int()
        label = torch.tensor(self.label.iloc[index, 3]).int()
        return sample, label

tensor_dataset = CustomDataset('./data/sample_01.csv')
dataset = DataLoader(tensor_dataset, batch_size=4, shuffle=True)

In [16]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

from torchvision.datasets import MNIST
import requests
download_root = './data/MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

100.0%


Extracting ./data/MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



100.0%

Extracting ./data/MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



100.0%


Extracting ./data/MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



100.0%

Extracting ./data/MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



## 모델 정의
- 모델 정의 : module 을 상속한 클래스 사용
    - 계층 (layer) : 모듈 또는 모듈을 구성하는 한개의 계층. convolution layer, linear layer 등
    - 모듈 (module) : 한 개 이상의 계층이 모여서 구성. 모듈이 모여 새로운 모듈을 만들 수도 있음
    - 모델 (model) : 최종적으로 원하는 네트워크. 한 개의 모듈이 모델이 될 수도 있음

### 단순 신경망 정의
- ```nn.Module```을 상속받지 않는 단순 모델

In [17]:
import torch.nn as nn

model = nn.Linear(in_features=1, out_features=1, bias=True)

### nn.Module()을 상속하여 정의하는 방법
- ```nn.Module```을 상속 시,
    - ```__init()``` : 모델에서 사용될 모듈 (nn.Linear, nn.Conv2d 등), 활성화 함수 등을 정의
    - ```forward()``` : 모델에서 실행되어야 하는 연산 정의

```
class MLP(nn.Module):
    def __init__(self, inputs):
        super(MLP, self).__init__()
        self.layer = Linear(inputs, 1)
        self.activation = Sigmoid()

    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X
```